## Sentence Retival
Copy this at the end of the other notebook


In [ ]:
# Where numToEval is max 360 for train
def evaluateSentenceRetrival(questionsList,documentsList, numToEval):
    correct = []
    wrong = []
    for i in range (0, numToEval):
        documents = documentsList[i]
        questions = questionsList[i]
        
        vector_documents, vectorizer = vectorize_documents(documents)
        analyze = vectorizer.build_analyzer()
        neighbours = process_neighbours(vector_documents)
        
        for j in range (0, len(questions)):
            text_query = questions[j]["question"]
            vector_query = vectorize_query(vectorizer, text_query)
            result_similarity, result_index  = closest_document(neighbours, vector_query)
            if result_index == int(questions[j]["answer_sentence"]):
                correct.append((i,j,result_index))
            else :
                wrong.append ((i,j,result_index))
    return (correct, wrong)

In [ ]:
(corSR, wrongSR) = evaluateSentenceRetrival(dev, devSents,len(dev))
print("Number Correct : " + str(len(corSR)))
print("Number incorrect: " + str(len(wrongSR)))
print ("Average correct : " + str((len(corSR) + 0.0) / (len(corSR)+len(wrongSR))))

In [ ]:
# Ran in the other docco. Getting about 60%

## NER
Tests to see if the required entity exists within the given taggings


In [ ]:
# For each question, evaluate if the answer is present as an entity

def evaluateNER(questionsList,documentsList, numToEval):
    correct = []
    wrong = []
    for i in range (0, numToEval):
        documents = documentsList[i]
        questions = questionsList[i]
        for j in range (0, len(questions)):
            answer = questionsList[i][j]["answer"]
            answerID = questionsList[i][j]["answer_sentence"]
            possAnswers = refineWordTags(taggedDev[i][answerID])
            inThere = False
            for possAnswer in possAnswers:
                if possAnswer[0] == answer:
                    inThere = True
                    break
            if inThere:
                correct.append((i,j, possAnswers))
            else :
                wrong.append((i,j, answer, possAnswers))
    return (correct, wrong)


In [ ]:
(corNER, wrongNER) = evaluateNER(dev, devSents,len(dev))

print("Number Correct : " + str(len(corNER)))
print("Number incorrect: " + str(len(wrongNER)))
print("Average correct : " + str((len(corNER) + 0.0) / (len(corNER)+len(wrongNER))))

In [ ]:
# Show the wrong taggings
for num in range (0, 10):
    (i,j, answer, possAnswers) = wrong[ num]
    print "ANSWER : " + answer
    for possAnswer in possAnswers:
        print possAnswer[0]
    print

## Answer Ranking
For each "correct" tagging above (ie tags where the answer is a member
Test the answer ranking


In [ ]:
# For each question, evaluate if the answer is present as an entity

def evaluateAnswerRanking(questionsList,documentsList, numToEval):
    correct = []
    wrong = []
    (corNER, wrongNER) = evaluateNER(questionsList,documentsList, numToEval)
    entityListsWithAnswer = corNER
    for (i,j,possAnswers) in entityListsWithAnswer:
        question = questionsList[i][j]["question"]
        answer =  questionsList[i][j]["answer"]
        answerPredicited = "" # TODO add Dereks part here
        if (answerPredicited == answer):
            correct.append((i,j))
        else :
            wrong.append((i,j,answerPredicited))
    return (correct, wrong)
    

In [ ]:
(corAns, wrongAns) = evaluateAnswerRanking(dev, devSents,len(dev))
print("Number Correct : " + str(len(corAns)))
print("Number incorrect: " + str(len(wrongAns)))
print ("Average correct : " + str((len(corAns) + 0.0) / (len(corAns)+len(wrongAns))))